## PeriPy framework

The purpose of this notebook is to introduce the design of PeriPy. This should enable the effective use of PeriPy and allow a user to extend the framework to solve problems other than those provided in the main distribution.

In [48]:
import os
import argparse
import cProfile
from io import StringIO
import pathlib

from pstats import SortKey, Stats
import numpy as np
import h5py
import matplotlib.pyplot as plt
import meshio

from peripy import Model
from peripy.integrators import VelocityVerletCL
from peripy.utilities import write_array
from peripy.utilities import read_array as read_model


In [49]:
absolute_path = os.path.abspath('')

# The .msh file is a point cloud. '175beam3620.msh' contains 3620 particles.
mesh_file = absolute_path + '/175beam3620.msh'

write_path_model = absolute_path + '/175beam3620.h5'
write_path_solutions = absolute_path


## Plot mesh

In [ ]:
mesh = meshio.read(mesh_file)
# Get coordinates, encoded as mesh points
coords = np.array(mesh.points, dtype=np.float64)
plt.scatter(coords[:, 0], coords[:, 2], s=10)
plt.axis('scaled')


## Define functions

In [50]:
def smooth_step_data(
        first_step, steps, start_value, final_value):
    """
    Apply a smooth 5th order polynomial as the displacement-time function for
    the simulation.

    :arg int start_time_step: The time step that the simulation starts from.
    :arg int steps: The number of steps in the simulation.
    :arg float start_value: The starting value of the displacement.
    :arg float final_value: The final value of the displacement.
    """
    alpha = np.zeros(first_step + steps)
    for current_time_step in range(first_step, first_step + steps):
        xi = (current_time_step - first_step) / steps
        alpha[current_time_step] = (start_value + (
            final_value - start_value) * xi**3 * (10 - 15 * xi + 6 * xi**2))
    return alpha


In [51]:
def is_tip(x):
    """
    Return a boolean list of tip types for each cartesian direction.

    Returns a boolean list, whose elements are a strings when the particle
    resides on a 'tip' to be measured for some displacement, velocity,
    acceleration, force or body_force in that cartesian direction. The value
    of the element of the list can be a string or an int, which is a flag for
    the tip type that the particle resides on. If a particle resides on more
    than one tip, then any of the list elements can be a tuple of tip types.

    :arg x: Particle coordinate array of size (3,).
    :type x: :class:`numpy.ndarray`

    :returns: A (3,) list of tip types.
    :rtype: List of (tuples of) None or string.
    """
    # Particle does not live on tip
    tip = [None, None, None]
    
    # Particle does live on a tip to me measured, in the centre of the beam
    if (x[0] > 0.0875 - 5.0e-3) and (x[0] < 0.0875 + 5.0e-3):
        if (x[2] > 0.025 - 5.0e-3) and (x[2] < 0.025 + 5.0e-3):
            # Measurements are made in the z direction
            tip[2] = 'deflection'
            
    # Measure the force applied at the boundaries, in the z direction
    if x[0] == 0.025 and x[2] == -0.01:
        tip[2] = 'force'
    if x[0] == 0.155 and x[2] == -0.01:
        tip[2] = 'force'
        
    # Measurement of the "crack mouth opening displacement" (CMOD)
    if x[0] == 0.125 and x[1] == 0.005 and x[2] == 0.005:
        tip[0] = 'CMOD_right'
    if x[0] == 0.055 and x[1] == 0.005 and x[2] == 0.005:
        tip[0] = 'CMOD_left'
        
    return tip


In [52]:
def is_density(x):
    """
    Return the density of the particle.

    :arg x: Particle coordinate array of size (3,).
    :type x: :class:`numpy.ndarray`

    :returns: density of concrete in [kg/m^3]
    :rtype: float
    """
    return 2346.0


In [53]:
def is_displacement_boundary(x):
    """
    Return a boolean list of displacement boundarys for each direction.

    Returns a (3,) boolean list, whose elements are:
        None where there is no boundary condition;
        -1 where the boundary is displacement loaded in negative direction;
        1 where the boundary is displacement loaded in positive direction;
        0 where the boundary is clamped;

    :arg x: Particle coordinate array of size (3,).
    :type x: :class:`numpy.ndarray`
    """
    # Particle does not live on a boundary
    bnd = [None, None, None]
    
    # Displacement controlled particles
    if x[0] == 0.085 and x[2] == 0.05:
        bnd[2] = -1
    if x[0] == 0.09 and x[2] == 0.05:
        bnd[2] = -1
    if x[0] == 0.095 and x[2] == 0.05:
        bnd[2] = -1
        
    # Clamped particles
    if (x[0] > 0.025 - 5e-3) and (x[0] < 0.025 + 5e-3) and (x[2] < -0.005):
        bnd[2] = 0
        bnd[1] = 0
    if (x[0] > 0.155 - 5e-3) and (x[0] < 0.155 + 5e-3) and (x[2] < -0.005):
        bnd[2] = 0
        bnd[1] = 0
        
    return bnd


## Constants

In [54]:

dx = 5e-3  # Grid resolution
horizon = dx * np.pi  # Peridynamic horizon

# Trilinear constitutive model
s_0 = 1.05e-4
s_1 = 6.90e-4
s_c = 5.56e-3
beta = 0.25
c = 2.32e18
c_1 = (beta * c * s_0 - c * s_0) / (s_1 - s_0)
c_2 = (- beta * c * s_0) / (s_c - s_1)
critical_stretch = [np.float64(s_0), np.float64(s_1), np.float64(s_c)]
bond_stiffness = [np.float64(c), np.float64(c_1), np.float64(c_2)]

dt = 1.3e-6  # Time step
damping = 2.5e6

## Boundary conditions

In [55]:
# The displacement boundary condition magnitudes increase with a 5th order
# polynomial so that the acceleration at the start and end of the
# simulation is zero. The displacement of 0.2mm is applied over
# 100,000 time-steps.
steps = 100000
applied_displacement = 2e-4
displacement_bc_array = smooth_step_data(0, steps, 0, applied_displacement)


## Solver (integrator)

In [57]:
integrator = VelocityVerletCL(dt=dt, damping=damping)


Device is AMD Radeon Pro 5500M Compute EngineGPU from AMD with a max of 24 compute units, 
a max of 256 work-items per work-group, 
a max work item dimensions of 3, 
a max work item sizes of [256, 256, 256],
and device local memory size is 65536 bytes. 


## Build model

In [59]:
model = Model(mesh_file,
              integrator=integrator,
              horizon=horizon,
              critical_stretch=critical_stretch,
              bond_stiffness=bond_stiffness,
              dimensions=3,
              volume_total=0.0004525,  # Total volume of the concrete beam
              is_density=is_density,
              is_displacement_boundary=is_displacement_boundary,
              is_tip=is_tip,
              write_path=write_path_model)

## Class: Model

In [ ]:
print(model.__dict__)

### family: numpy.ndarray
* Size: (nnodes, )
* dtype: int
* Explanation: number of family members per node
* Rename to n_family_members

In [65]:
print('type:', type(model.family))

type: <class 'numpy.ndarray'>


### initial_connectivity: tuple(numpy.ndarray, numpy.ndarray)
* Size: no strict dimensions (nnodes, n_family_members) or (nnodes, max_neighbours)?
* dtype: int
* Explanation: bond connectivity matrix
* What is the difference between initial_connectivity and connectivity?
* Rename to connectivity_matrix

In [73]:
print('type:', type(model.initial_connectivity))

type: <class 'tuple'>


### bc_types: numpy.ndarray
* Applied displacements (displacement controlled)
* Size: (nnodes, dof)
* dtype: int
* Explanation: A value of 1 if the node is loaded in the positive cartesian directions, a value of -1 if the node is loaded in the negative caterisan direction, and a value of 0 if it is constrained / for no boundary conditions??? Definition is different in `Model.__init__` and `Model._set_boundary_conditions`. In the examples, bc_types is used a flag and no direction is assigned. The direction (positive or negative) is applied in bc_values.

In [60]:
print('type:', type(model.bc_types))
test_bc_types = model.bc_types

type: <class 'numpy.ndarray'>


### bc_values: numpy.ndarray
* Applied displacements (displacement controlled)
* Size: (nnodes, dof)
* dtype: float
* Explanation: magnitude of the boundary condition (value of appplied displacement)

In [61]:
print('type:', type(model.bc_values))
test_bc_values = model.bc_values

type: <class 'numpy.ndarray'>


### constraints

Constraints are applied by setting the value of bc_types to 1 and then setting the value of bc_values to 0. This is a potentially confusing method of applying constraints. Furthermore, this method makes it harder to visualise the constrained nodes.

### force_bc_types / force_bc_values

* Identical to bc_types and bc_values
* Applied forces (load controlled)


### tips_types: dict
* displacement
* velocity
* acceleration
* force
* body_force

In [ ]:
print('type:', type(model.tip_types))
print('[(node_id, dof)]')
print('\nCMOD_left:', model.tip_types['CMOD_left'])
print('\nCMOD_right:', model.tip_types['CMOD_right'])
print('\ndeflection:', model.tip_types['deflection'])
print('\nforce:', model.tip_types['force'])

## Visualise the constitutive model

In [ ]:
# Visualise the particular constitutive model being used
bond_stiffness, critical_stretch, plus_cs, *_ = model._set_damage_model(
    bond_stiffness, critical_stretch)
plt.plot(
    [0, critical_stretch[0]],
    [0, critical_stretch[0] * bond_stiffness[0] + plus_cs[0]],
    color='k')
plt.plot(
    [critical_stretch[0], critical_stretch[1]],
    [critical_stretch[0] * bond_stiffness[0],
        critical_stretch[1] * bond_stiffness[1] + plus_cs[1]],
    color='k')
plt.plot(
    [critical_stretch[1], critical_stretch[2]],
    [critical_stretch[1] * bond_stiffness[1] + plus_cs[1],
        critical_stretch[2] * bond_stiffness[2] + plus_cs[2]],
    color='k')
plt.xlabel('Bond stretch (dimensionless)')
plt.ylabel('Bond force [N/m^6]')
plt.title('Trilinear constitutive model')


## Run the simulation

In [ ]:
(u, damage, connectivity, f, ud, data) = model.simulate(bond_stiffness=bond_stiffness,
                                                        critical_stretch=critical_stretch,
                                                        steps=steps,
                                                        displacement_bc_magnitudes=displacement_bc_array,
                                                        write_mesh=5000,  # write to mesh every 5000 time steps
                                                        write_data=100)   # write to data every 100 time steps


## Write the numerical load-CMOD data to disk

In [ ]:
force = np.array(data['force']['body_force']) / 1000
left_displacement = 1000. * np.array(data['CMOD_left']['displacement'])
right_displacement = 1000. * np.array(data['CMOD_right']['displacement'])
CMOD = np.subtract(right_displacement, left_displacement)

# Write load-CMOD data to disk
try:
    write_array(write_path_solutions + "/data.h5", "force", np.array(force))
    write_array(write_path_solutions + "/data.h5", "CMOD", np.array(CMOD))
except OSError:  # data.h5 already exists
    os.remove(write_path_solutions + "/data.h5")
    write_array(write_path_solutions + "/data.h5", "force", np.array(force))
    write_array(write_path_solutions + "/data.h5", "CMOD", np.array(CMOD))


## Plot the experimental load-CMOD data, verification data, and numerical output

In [ ]:
# Plot the experimental data
exp_data_path = absolute_path + "/experimental_data.h5"
exp_data = h5py.File(exp_data_path, 'r')
exp_load_CMOD = exp_data['load_CMOD']
exp_CMOD = exp_load_CMOD[0, 0:20000]
exp_load_mean = exp_load_CMOD[1, 0:20000]
exp_load_min = exp_load_CMOD[2, 0:20000]
exp_load_max = exp_load_CMOD[3, 0:20000]
plt.plot(exp_CMOD, exp_load_mean, color=(0.8, 0.8, 0.8), label='Experimental')
plt.fill_between(exp_CMOD, exp_load_min, exp_load_max, color=(0.8, 0.8, 0.8))

# Plot the verification data
ver_data_path = absolute_path + "/verification_data.h5"
ver_data = h5py.File(ver_data_path, 'r')
ver_load_CMOD = ver_data['load_CMOD']
ver_load = ver_load_CMOD[0, 0:499] / 1000
ver_CMOD = ver_load_CMOD[1, 0:499]
plt.plot(ver_CMOD, ver_load, 'tab:orange', label='Verification data')

# Plot the numerical data
force = read_model(absolute_path + "/data.h5", "force")
CMOD = read_model(absolute_path + "/data.h5", "CMOD")
plt.plot(CMOD, -force, label='Numerical')

# Format the figure
plt.xlabel('CMOD [mm]')
plt.ylabel('Force [kN]')
plt.grid(True)
axes = plt.gca()
axes.set_xlim([0, .3])
axes.set_ylim([0, 5.5])
axes.tick_params(direction='in')
plt.legend()


## Visualise the damage

In [ ]:
# Use PyVista